# Pembelajaran Mesin: Tugas Besar 2 (Model Selection and Prediction)

Kaenova Mahendra Auditama (1301190324)  
Moh Adi Ikfini M. (1301194160)  
IF-43-02  

  
Pada kodingan ini, kami mencoba untuk membuat sebuah model yang akan digunakan untuk membuat prediksi pada suatu data. Data yang diberikan merupakan data ketertarikan pelanggan untuk membeli kendaraan baru. Kami diminta untuk menggunakan metode supervised learning untuk melakukan prediksi tersebut. Kami akan mencoba melakukan prediksi menggunakan training data dan testing data dengan data yang imbalance.

# Pre-Requisite

## Import Library

In [96]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pickle
import random
import math
import sklearn
from sklearn.preprocessing import MinMaxScaler
from urllib.request import urlopen
from scipy.stats import chi2_contingency
import numpy as np
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import imblearn
import sklearn
# Seeding for predictable output
tf.random.set_seed(1)
np.random.seed(1)

## Loading Data

### Saving All Training Data and Testing Data

In [90]:
import bz2
import pickle
import _pickle as cPickle
# ref: https://betterprogramming.pub/load-fast-load-big-with-compressed-pickles-5f311584507e
def compressed_pickle(title, data):
    with bz2.BZ2File("./" +title + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data
# Load training and testing data
!wget https://raw.githubusercontent.com/kaenova/static_raw_file/main/Malin2_training_testing_preprocessed.pbz2
training_testing_data = decompress_pickle("./Malin2_training_testing_preprocessed.pbz2")

--2021-12-17 05:46:13--  https://raw.githubusercontent.com/kaenova/static_raw_file/main/Malin2_training_testing_preprocessed.pbz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21639656 (21M) [application/octet-stream]
Saving to: ‘Malin2_training_testing_preprocessed.pbz2.2’

Malin2_training_tes 100%[===================>]  20.64M   115MB/s    in 0.2s    

2021-12-17 05:46:13 (115 MB/s) - ‘Malin2_training_testing_preprocessed.pbz2.2’ saved [21639656/21639656]



In [98]:
# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
    with bz2.BZ2File("./" +title + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)

In [99]:
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [100]:
# # Save training and testing data
# compressed_pickle("training_testing_preprocessed", training_testing_data)

In [101]:
# Load training and testing data
!wget https://raw.githubusercontent.com/kaenova/static_raw_file/main/Malin2_training_testing_preprocessed.pbz2
training_testing_data = decompress_pickle("./Malin2_training_testing_preprocessed.pbz2")

--2021-12-17 06:07:47--  https://raw.githubusercontent.com/kaenova/static_raw_file/main/Malin2_training_testing_preprocessed.pbz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21639656 (21M) [application/octet-stream]
Saving to: ‘Malin2_training_testing_preprocessed.pbz2.4’

Malin2_training_tes 100%[===================>]  20.64M   102MB/s    in 0.2s    

2021-12-17 06:07:48 (102 MB/s) - ‘Malin2_training_testing_preprocessed.pbz2.4’ saved [21639656/21639656]



# Naive Bayes from Scratch

In [125]:
class GaussianNB:
    '''
    Karena mepet, saya tidak melakukan handling jika yang tidak ada kombinasinya
    '''
    # ref: https://iq.opengenus.org/gaussian-naive-bayes
    kelas = None # np.array
    jumlah_kelas = 0 # int
    jumlah_baris = 0 # int
    jumlah_fitur = 0 # int

    rataan = None # np.array (n_fitur, n_kelas)
    varian = None # np.array (n_fitur, n_kelas)
    target = None # np.array (n_kelas,)

    def __init__(self):
        print("[INFO] Model terinisialisasi!")

    def fit(self, X:np.array, y:np.array):
        self.kelas = np.unique(y)
        self.jumlah_kelas = len(self.kelas)
        self.jumlah_fitur = len(X[0])
        self.__setup_rataan_varian_target__(X, y)
        print("[INFO] Model sudah berhasil belajar")

    def predict(self, X:np.array) -> np.array:
        hasil = np.zeros((len(X), ))
        for i in range(len(X)):
            hasil[i] = self.kelas[self.__get_kelas__(X[i])]
        return hasil

    def __setup_rataan_varian_target__(self, X: np.array, y:np.array):
        '''
        Hitung Probabilitas Fitur terhadap Target (P(Xi, Xi+1, Xi+2, ... Xn))
        Hitung Probabilitas Target (P(yi))
        ''' 
        ## Setup group by 
        kelas = {}
        for i in self.kelas:
            kelas[i] = []
        for i in range(len(y)):
            if y[i] in kelas.keys():
                kelas[y[i]].append(i)
        
        # Setup probabilitas fitur terhadap target
        self.rataan = np.array([np.zeros((self.jumlah_fitur, )) for i in range(self.jumlah_kelas)])
        self.varian = np.array([np.zeros((self.jumlah_fitur, )) for i in range(self.jumlah_kelas)])
        j = 0
        for i in kelas.keys():
            sum = np.sum(X[kelas[i]], axis=0)
            self.rataan[j] = sum/len(kelas[i])
            var = np.var(X[kelas[i]], axis=0)
            self.varian[j] = var
            j += 1

        # Setup probabilitas target
        self.target = np.zeros((self.jumlah_kelas,))
        j = 0
        for i in kelas.keys():
            self.target[j] = len(kelas[i]) / len(y)
            j += 1
    
    def __get_kelas__(self, X: np.array) -> int:
        '''
        X (n_fitur, )
        '''
        prob = []
        for i in range(len(self.kelas)):
            prob_kelas = np.log(self.target[i])
            prob_fitur = np.sum(np.log(np.exp((-1/2)*((X-self.rataan[i])**2) / (2 * self.varian[i])) / np.sqrt(2 * np.pi * self.varian[i])))
            prob.append(prob_kelas + prob_fitur)
        return np.argmax(prob)

# Modelling

## Model Exploration (on "Hypothesis" Best Feature)


### Decssion Tree

In [ ]:
# Template untuk modeling dengan k folds
training_data_args = 'best_one_hot_oridnalOnTarget' #@param ["best_one_hot", "all_one_hot", "all_ordinal", "best_one_hot_oridnalOnTarget"]
training_data_fold = training_testing_data[training_data_args]
f1 = []
model_hist = []
print("=== Train using Folds")
for i in training_data_fold.keys():
    if type(i) == int:
        print("Folds Number:", i+1)
        X_train, X_test, y_train, y_test = training_data_fold[i]["X_train"], training_data_fold[i]["X_test"], training_data_fold[i]["y_train"], training_data_fold[i]["y_test"]
        # Masukkin Model
        model = sklearn.tree.DecisionTreeClassifier().fit(X_train, y_train)
        # Append akurasi ke variable 'akurasi'
        y_test_out = model.predict(X_test)
        print(sklearn.metrics.classification_report(y_test, y_test_out))
        f1_scr = sklearn.metrics.f1_score(y_test, y_test_out)
        print("F1:", f1_scr)
        f1.append(f1_scr)
        model_hist.append(model)
        
print("F1 rata-rata model:",np.mean(f1))
# Coba validasi data testing
## Ambil model terbaik
model = model_hist[np.argmax(np.array(f1))]
print('\n\n')
print("=== Checking on Data Testing")
## Prediksikan ke data testing
predicted = model.predict(training_testing_data[training_data_args]["Test"]["X"])
## Lihat akurasi data testing
print(sklearn.metrics.classification_report(training_testing_data[training_data_args]["Test"]["y"], predicted))

=== Train using Folds
Folds Number: 1
              precision    recall  f1-score   support

           0       0.97      0.59      0.73     31671
           1       0.66      0.98      0.79     25336

    accuracy                           0.76     57007
   macro avg       0.81      0.78      0.76     57007
weighted avg       0.83      0.76      0.76     57007

F1: 0.7850081718792149
Folds Number: 2
              precision    recall  f1-score   support

           0       0.97      0.59      0.74     31671
           1       0.66      0.98      0.79     25336

    accuracy                           0.76     57007
   macro avg       0.81      0.78      0.76     57007
weighted avg       0.83      0.76      0.76     57007

F1: 0.7860246772607605
Folds Number: 3
              precision    recall  f1-score   support

           0       0.97      0.59      0.73     31671
           1       0.66      0.98      0.79     25336

    accuracy                           0.76     57007
   macro avg

### Gaussian NB

In [126]:
# Template untuk modeling dengan k folds
training_data_args = 'best_one_hot_oridnalOnTarget' #@param ["best_one_hot", "all_one_hot", "all_ordinal", "best_one_hot_oridnalOnTarget"]
training_data_fold = training_testing_data[training_data_args]
f1 = []
model_hist = []
print("=== Train using Folds")
for i in training_data_fold.keys():
    if type(i) == int:
        print("Folds Number:", i+1)
        X_train, X_test, y_train, y_test = training_data_fold[i]["X_train"], training_data_fold[i]["X_test"], training_data_fold[i]["y_train"], training_data_fold[i]["y_test"]
        # Masukkin Model
        model = GaussianNB()
        model.fit(X_train, y_train)
        # Append akurasi ke variable 'akurasi'
        y_test_out = model.predict(X_test)
        print(sklearn.metrics.classification_report(y_test, y_test_out))
        f1_scr = sklearn.metrics.f1_score(y_test, y_test_out)
        print("F1:", f1_scr)
        f1.append(f1_scr)
        model_hist.append(model)
        
print("F1 rata-rata model:",np.mean(f1))
# Coba validasi data testing
## Ambil model terbaik
model = model_hist[np.argmax(np.array(f1))]
model_naive = model_hist[np.argmax(np.array(f1))]
print('\n\n')
print("=== Checking on Data Testing")
## Prediksikan ke data testing
predicted = model.predict(training_testing_data[training_data_args]["Test"]["X"])
## Lihat akurasi data testing
print(sklearn.metrics.classification_report(training_testing_data[training_data_args]["Test"]["y"], predicted))

=== Train using Folds
Folds Number: 1
[INFO] Model terinisialisasi!
[INFO] Model sudah berhasil belajar
              precision    recall  f1-score   support

           0       0.97      0.59      0.73     31671
           1       0.66      0.98      0.79     25336

    accuracy                           0.76     57007
   macro avg       0.81      0.78      0.76     57007
weighted avg       0.83      0.76      0.76     57007

F1: 0.7850081718792149
Folds Number: 2
[INFO] Model terinisialisasi!
[INFO] Model sudah berhasil belajar
              precision    recall  f1-score   support

           0       0.97      0.59      0.74     31671
           1       0.66      0.98      0.79     25336

    accuracy                           0.76     57007
   macro avg       0.81      0.78      0.76     57007
weighted avg       0.83      0.76      0.76     57007

F1: 0.7860246772607605
Folds Number: 3
[INFO] Model terinisialisasi!
[INFO] Model sudah berhasil belajar
              precision    recal

### Artificial Neural Network

In [ ]:
# Template untuk modeling dengan k folds
training_data_args = 'best_one_hot' #@param ["best_one_hot", "all_one_hot", "all_ordinal", "best_one_hot_oridnalOnTarget"]
training_data_fold = training_testing_data[training_data_args]
f1 = []
model_hist = []
for i in training_data_fold.keys():
    if type(i) == int:
        print("Folds Number:", i+1)
        X_train, X_test, y_train, y_test = training_data_fold[i]["X_train"], training_data_fold[i]["X_test"], training_data_fold[i]["y_train"], training_data_fold[i]["y_test"]
        # Masukkin Model
        model = Sequential()
        model.add(Dense(12, input_dim=len(X_train[0]), activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(len(y_train[0]), activation='sigmoid')) 
        model.compile(optimizer="Adam", loss="binary_crossentropy", 
                      metrics=["accuracy",
                               "categorical_accuracy",
                               tf.keras.metrics.Precision(),
                               tf.keras.metrics.Recall()
                               ]
                    )
        model.fit(X_train, y_train, epochs=5, batch_size=20)
        # Append akurasi ke variable 'akurasi'
        y_test_out = model.predict(X_test)
        print(sklearn.metrics.classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_out, axis=1)))
        f1_scr = sklearn.metrics.f1_score(np.argmax(y_test, axis=1), np.argmax(y_test_out, axis=1))
        print("F1:", f1_scr)
        f1.append(f1_scr)
        model_hist.append(model)
        # Append akurasi ke variable 'akurasi'
 
print("F1 rata-rata model:",np.mean(f1))
# Coba validasi data testing
## Ambil model terbaik
model = model_hist[np.argmax(np.array(f1))]
print('\n\n')
print("=== Checking on Data Testing")
## Prediksikan ke data testing
predicted = model.predict(training_testing_data[training_data_args]["Test"]["X"])
## Lihat akurasi data testing
print(sklearn.metrics.classification_report(np.argmax(training_testing_data[training_data_args]["Test"]["y"], axis=1), np.argmax(predicted, axis=1)))

Folds Number: 1
Epoch 1/5
11402/11402 [==============================] - 26s 2ms/step - loss: 0.4594 - accuracy: 0.7558 - categorical_accuracy: 0.7558 - precision_6: 0.7521 - recall_2: 0.7612
Epoch 2/5
11402/11402 [==============================] - 24s 2ms/step - loss: 0.4520 - accuracy: 0.7629 - categorical_accuracy: 0.7629 - precision_6: 0.7629 - recall_2: 0.7629
Epoch 3/5
11402/11402 [==============================] - 24s 2ms/step - loss: 0.4520 - accuracy: 0.7629 - categorical_accuracy: 0.7629 - precision_6: 0.7629 - recall_2: 0.7629
Epoch 4/5
11402/11402 [==============================] - 24s 2ms/step - loss: 0.4520 - accuracy: 0.7629 - categorical_accuracy: 0.7629 - precision_6: 0.7629 - recall_2: 0.7629
Epoch 5/5
11402/11402 [==============================] - 23s 2ms/step - loss: 0.4520 - accuracy: 0.7629 - categorical_accuracy: 0.7629 - precision_6: 0.7629 - recall_2: 0.7629
              precision    recall  f1-score   support

           0       0.97      0.59      0.73     3

## Model Exploration (on All Feature)


### Decssion Tree

In [ ]:
# Template untuk modeling dengan k folds
training_data_args = 'all_one_hot' #@param ["best_one_hot", "all_one_hot", "all_ordinal", "best_one_hot_oridnalOnTarget"]
training_data_fold = training_testing_data[training_data_args]
f1 = []
model_hist = []
print("=== Train using Folds")
for i in training_data_fold.keys():
    if type(i) == int:
        print("Folds Number:", i+1)
        X_train, X_test, y_train, y_test = training_data_fold[i]["X_train"], training_data_fold[i]["X_test"], training_data_fold[i]["y_train"], training_data_fold[i]["y_test"]
        # Masukkin Model
        model = sklearn.tree.DecisionTreeClassifier().fit(X_train, y_train)
        # Append akurasi ke variable 'akurasi'
        y_test_out = model.predict(X_test)
        print(sklearn.metrics.classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_out, axis=1)))
        f1_scr = sklearn.metrics.f1_score(np.argmax(y_test, axis=1), np.argmax(y_test_out, axis=1))
        print("F1:", f1_scr)
        f1.append(f1_scr)
        model_hist.append(model)
        
print("F1 rata-rata model:",np.mean(f1))
# Coba validasi data testing
## Ambil model terbaik
model = model_hist[np.argmax(np.array(f1))]
print('\n\n')
print("=== Checking on Data Testing")
## Prediksikan ke data testing
predicted = model.predict(training_testing_data[training_data_args]["Test"]["X"])
## Lihat akurasi data testing
print(sklearn.metrics.classification_report(np.argmax(training_testing_data[training_data_args]["Test"]["y"], axis=1), np.argmax(predicted,axis=1)))

=== Train using Folds
Folds Number: 1
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     31671
           1       0.87      0.87      0.87     25336

    accuracy                           0.88     57007
   macro avg       0.88      0.88      0.88     57007
weighted avg       0.88      0.88      0.88     57007

F1: 0.8700685513344791
Folds Number: 2
              precision    recall  f1-score   support

           0       0.89      0.90      0.89     31671
           1       0.87      0.87      0.87     25336

    accuracy                           0.88     57007
   macro avg       0.88      0.88      0.88     57007
weighted avg       0.88      0.88      0.88     57007

F1: 0.8676022929432695
Folds Number: 3
              precision    recall  f1-score   support

           0       0.89      0.89      0.89     31671
           1       0.87      0.87      0.87     25336

    accuracy                           0.88     57007
   macro avg

### Artificial Neural Network

In [ ]:
# Template untuk modeling dengan k folds
training_data_args = 'all_one_hot' #@param ["best_one_hot", "all_one_hot", "all_ordinal", "best_one_hot_oridnalOnTarget"]
training_data_fold = training_testing_data[training_data_args]
f1 = []
model_hist = []
for i in training_data_fold.keys():
    if type(i) == int:
        print("Folds Number:", i+1)
        X_train, X_test, y_train, y_test = training_data_fold[i]["X_train"], training_data_fold[i]["X_test"], training_data_fold[i]["y_train"], training_data_fold[i]["y_test"]
        # Masukkin Model
        model = Sequential()
        model.add(Dense(12, input_dim=len(X_train[0]), activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(len(y_train[0]), activation='sigmoid')) 
        model.compile(optimizer="Adam", loss="binary_crossentropy", 
                      metrics=["accuracy",
                               "categorical_accuracy",
                               tf.keras.metrics.Precision(),
                               tf.keras.metrics.Recall()
                               ]
                    )
        model.fit(X_train, y_train, epochs=5, batch_size=20)
        # Append akurasi ke variable 'akurasi'
        y_test_out = model.predict(X_test)
        print(sklearn.metrics.classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_out, axis=1)))
        f1_scr = sklearn.metrics.f1_score(np.argmax(y_test, axis=1), np.argmax(y_test_out, axis=1))
        print("F1:", f1_scr)
        f1.append(f1_scr)
        model_hist.append(model)
        # Append akurasi ke variable 'akurasi'
 
print("F1 rata-rata model:",np.mean(f1))
# Coba validasi data testing
## Ambil model terbaik
model = model_hist[np.argmax(np.array(f1))]
print('\n\n')
print("=== Checking on Data Testing")
## Prediksikan ke data testing
predicted = model.predict(training_testing_data[training_data_args]["Test"]["X"])
## Lihat akurasi data testing
print(sklearn.metrics.classification_report(np.argmax(training_testing_data[training_data_args]["Test"]["y"], axis=1), np.argmax(predicted, axis=1)))

Folds Number: 1
Epoch 1/5
11402/11402 [==============================] - 32s 3ms/step - loss: 0.4289 - accuracy: 0.7852 - categorical_accuracy: 0.7852 - precision_11: 0.7843 - recall_7: 0.7838
Epoch 2/5
11402/11402 [==============================] - 25s 2ms/step - loss: 0.4198 - accuracy: 0.7905 - categorical_accuracy: 0.7905 - precision_11: 0.7901 - recall_7: 0.7907
Epoch 3/5
11402/11402 [==============================] - 25s 2ms/step - loss: 0.4179 - accuracy: 0.7905 - categorical_accuracy: 0.7905 - precision_11: 0.7903 - recall_7: 0.7905
Epoch 4/5
11402/11402 [==============================] - 25s 2ms/step - loss: 0.4165 - accuracy: 0.7920 - categorical_accuracy: 0.7920 - precision_11: 0.7918 - recall_7: 0.7921
Epoch 5/5
11402/11402 [==============================] - 24s 2ms/step - loss: 0.4151 - accuracy: 0.7927 - categorical_accuracy: 0.7927 - precision_11: 0.7926 - recall_7: 0.7929
              precision    recall  f1-score   support

           0       0.90      0.70      0.79 

# Got the Best Model. Now Predict the Test Data

Naive Bayes It is!

In [128]:
predicted = model_naive.predict(training_testing_data[training_data_args]["Test"]["X"])

Let's append it to Raw DataTest

In [129]:
df_test = pd.read_csv("https://raw.githubusercontent.com/kaenova/Malin_Tubes1/main/data/raw/kendaraan_test.csv")
df_test_final = df_test.copy()
df_test_final["Prediksi"] = predicted
df_test_final[df_test_final["Prediksi"] == 1]
df_test_final.to_csv("kendaraan_test_predicted.csv")

# Having Fun

In [ ]:
def predict(X:np.array) -> np.array:
  return np.zeros((len(X),))

In [ ]:
predicted_fun = predict(training_testing_data[training_data_args]["Test"]["X"])
sklearn.metrics.accuracy_score(training_testing_data[training_data_args]["Test"]["y"], predicted_fun)

0.8769705493398267

Accuracy does not good for this data ^^. But if you really want to score based on accuracy. Then there you go! Model npzeros